In [1]:
import pandas as pd
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

/Users/Sam/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
os.chdir('/Users/Sam/ds/metis/metisgh/sf16_ds4/local/Project3')

In [3]:
df = pd.read_csv('default_of_credit_card_clients.csv')
df = df.dropna()

In [4]:
df = df.drop('ID', axis = 1)

In [5]:
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000.0,2.0,2.0,1.0,24.0,2.0,2.0,-1.0,-1.0,-2.0,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1.0
1,120000.0,2.0,2.0,2.0,26.0,-1.0,2.0,0.0,0.0,0.0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1.0
2,90000.0,2.0,2.0,2.0,34.0,0.0,0.0,0.0,0.0,0.0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0.0
3,50000.0,2.0,2.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0.0
4,50000.0,1.0,2.0,1.0,57.0,-1.0,0.0,-1.0,0.0,0.0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0.0


In [6]:
df['default payment next month'] = df['default payment next month'].replace(to_replace=0, value="Paid")
df['default payment next month'] = df['default payment next month'].replace(to_replace=1, value="Default")
target_names = ["Default", "Paid"]

In [7]:
X = df.drop('default payment next month', axis = 1)
target = df['default payment next month']

In [8]:
X = df[['LIMIT_BAL', 'SEX', 'EDUCATION','MARRIAGE','AGE']]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state = 42)

In [10]:
modelknn = KNeighborsClassifier()
knnfit = modelknn.fit(X_train, y_train)
knny_pred = knnfit.predict(X_test)

In [11]:
accuracy_score(y_test, knny_pred)

0.74277777777777776

# Feature Scaling Limit Balance

In [12]:
df['LIMIT_BAL'] = df['LIMIT_BAL']/1000

In [50]:
def knnclassonefeature(name):
    arr = []
    X = df[[name]]
    X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state = 42)
    modelknn = KNeighborsClassifier()
    knnfit = modelknn.fit(X_train, y_train)
    knny_pred = knnfit.predict(X_test)
    arr.append(accuracy_score(y_test, knny_pred))
    arr.append(recall_score(y_test, knny_pred, labels = ["Paid", "Default"], pos_label= "Paid"))
    arr.append(precision_score(y_test, knny_pred, labels = ["Paid", "Default"], pos_label = "Default"))
    return arr

In [51]:
colnames = df.columns.tolist()
colnames.pop()

'default payment next month'

In [52]:
for title in colnames:
    print title
    print knnclassonefeature(title)

LIMIT_BAL
[0.75944444444444448, 0.94161931818181821, 0.33387358184764993]
SEX
[0.78222222222222226, 1.0, 0.0]
EDUCATION
[0.78222222222222226, 1.0, 0.0]
MARRIAGE
[0.78222222222222226, 1.0, 0.0]
AGE
[0.74577777777777776, 0.93380681818181821, 0.22847682119205298]
PAY_0
[0.78177777777777779, 0.99687499999999996, 0.45000000000000001]
PAY_2
[0.79044444444444439, 0.92144886363636369, 0.53135593220338984]
PAY_3
[0.43688888888888888, 0.35355113636363639, 0.24074074074074073]
PAY_4
[0.78566666666666662, 0.93053977272727273, 0.51536174430128845]
PAY_5
[0.7921111111111111, 0.94318181818181823, 0.55005624296962885]
PAY_6
[0.78666666666666663, 0.93948863636363633, 0.52242152466367708]
BILL_AMT1
[0.74211111111111117, 0.92173295454545456, 0.25641025641025639]
BILL_AMT2
[0.74522222222222223, 0.92982954545454544, 0.24580152671755726]
BILL_AMT3
[0.74277777777777776, 0.9285511363636364, 0.22734254992319508]
BILL_AMT4
[0.74433333333333329, 0.93110795454545459, 0.2289348171701113]
BILL_AMT5
[0.7484444444444

In [ ]:


print knnclassonefeature('LIMIT_BAL')

In [ ]:
recall_score()
precision_score()